# Вебинары курса Рекомендательные системы

1. Введение, примеры задач, бизнес- и ML-метрики
2. Рекомендательные системы, основанные на диалоговом подходе.
3. Коллаборативная фильтрация: ALS.
4. Рекомендательные системы на основе контента. Поиск похожих товаров и пользователей. LightFM моедль. 
5. Ранжирование и гибридные рекомендательные системы
6. Двухуровневые модели рекомендаций. Рекомендательные системы в бизнесе.
7. Uplift
8. Консультация к финальному проекту. Разбор ДЗ.



ДЗ:
1. На "отлично" - сдать все задачи
2. Сдавать в виде ссылок на git (не pull), на colab (не забыть расшарить права на чтение)
3. Сроки - лоялно (главное успеть к концу)

# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

Николай Котенко
1. ВШЭ: информационная бизнес-аналитика
2. Тинькофф: аналитик-разработчик
3. Сбер: Digital audit

-----

# 1. Введение и примеры задач

[RecSys everywhere](https://drive.google.com/file/d/1o9qF90P_AGoyYDn_aX3ph3kXnBQ55q7M/view?usp=sharing)

----

[Skills](https://drive.google.com/file/d/16n2C1NQ5UJR9lHaPhBanQJ-vCl38xQee/view?usp=sharing)

---

##  Постановка задачи

*Задача рекомендаций* - предложить пользователю товары, которые ему "понравятся"

Компоненты:
    - Пользователи (users)
    - Товары (items)
    - Взаимодействия (interactions) - по ним понимаем, что товар "понравился"

---

# Why no NN?

https://arxiv.org/pdf/1907.06902.pdf

https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation

________________________________________________________________________________

## Пользователи VS Товары

[ex_user_items](https://drive.google.com/file/d/1m-nBz2aGgozqqmgD5mDxdUT6yOr4D4H_/view?usp=sharing)


**Кейс**: определить в каждой ситуации, что является пользователем, а что - товаром
    
    1. На сайте Кинопоиска порекомендовать интересные фильмы
    2. На сайте hh.ru подобрать лучшие вакансии для соискателя вакансий
    3. На сайте hh.ru найти похожие вакансии на ту, которую сейчас просматривает пользователь ("вам также может понравиться")
    3. На сайте hh.ru подобрать лучших кандидатов для работодателя
    
*Пример ответа в чате:*
    1. Пользователь (П) - фильм, Товар (T) - фильм
    2. П - работодатель, Т - соискатель вакансий

---

## Взаимодействия --> "понравился товар"

[ex_interactions](https://drive.google.com/file/d/1KjphiYNrkcK5YcRdEnsaDK0dOPdGT0Ct/view?usp=sharing)


---

## В чем отличие от классификации?

In [89]:
import pandas as pd
import numpy as np
import math

In [35]:
pd.DataFrame([['Иван', 'Хлеб Бородинский', 1],
             ['Иван', 'Хлеб Белый', 0],
             ['Василий', 'Йогурт Epica', 1]], columns=['user', 'item', 'purchase_fact'])

,user,item,purchase_fact
0,Иван,Хлеб Бородинский,1
1,Иван,Хлеб Белый,0
2,Василий,Йогурт Epica,1


Постойте, ведь можно добавить фичи на user (средний чек, кол-во покупок в категориях и т.д), 
на item (цена, кол-во продаж в неделю и т.l), и решать задачу классификации. В чем отличие RecSys от классификации?

 - Много предсказаний для 1 user (extreme classification)
 - Гораздо больший объем данных: 100K users, 10K items --> 1B предсказаний
 - Большинство товаров user никогда не видел --> Не взаимодействовал --> 0 не значит "не понравился"
 - Нет явного таргета. Не понятно, что значит "(не) понравился"
 - Feedback loop
 - Иногда важен порядок рекомендаций

In [36]:
pd.DataFrame([['Иван', 'Хлеб Бородинский', 1],
              ['Иван', 'Хлеб Белый', 0],
              ['Иван', 'Йогурт Epica', "?"],
              ['Василий', 'Хлеб Бородинский', "?"],
              ['Василий', 'Хлеб Белый', "?"],
              ['Василий', 'Йогурт Epica', 1]], columns=['user', 'item', 'purchase_fact'])

,user,item,purchase_fact
0,Иван,Хлеб Бородинский,1
1,Иван,Хлеб Белый,0
2,Иван,Йогурт Epica,?
3,Василий,Хлеб Бородинский,?
4,Василий,Хлеб Белый,?
5,Василий,Йогурт Epica,1


----

## Основные типы задач:
    
**Рекомендация топ-К товаров**: Дан список товаров. Рекомендовать пользователю K товаров, которые ему понравятся
    - e-mail рассылки (М.Видео, Hoff, Пятерочка)
    - push-уведомления (Delivery Club, HeadHunter, Виктория)
    - Рекомендации в отдельном окне на сайте (vk, okko)

**Ранжирование товаров**: Дан список товаров. Нужно его отранжировать в порядке убывания интереса для пользователя
    - Ранжирование каталога товаров (Wildberries, Lamoda)
    - Ранжирование ленты (vk, Я.Дзен)
    - Ранжирование поисковой выдачи (Яндекс, Гугл)
    - Ранжирование "каруселей" товаров (Delivery Club, Я.Еда)
    
**Поиск похожих товаров**: Дан 1 товар. Нужно найти наиболее похожие на него товары
    - "Вам также может понравиться" (Ozon, Циан)
    - Похожим пользователям понравилось (vk)
    - Вы можете быть знакомы (vk, facebook)
    
**Рекомендация дополнителдьного товара**. Дан 1 товар. Найти товары, которые покупают вместе с этим товаром
    - С этим товаром часто покупают (Я.Маркет, Ozon, М.Видео)
    
   
---
На 5 вебинаре мы разберем системы ранжирования

---

# 2. ML-mетрики качества

*Хороши ли рекомендации, предлагаемые моделью?*  
Обычно считаются для каждого юзера, затем усредняются по юзерам

Представим, что в магазине всего 10 товаров

In [37]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [38]:
len(recommended_list)

10

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [39]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    hit_rate_at_k = (flags.sum() > 0) * 1
    
    
    return hit_rate_at_k

In [40]:
hit_rate(recommended_list, bought_list)

1

In [41]:
hit_rate_at_k(recommended_list, bought_list, k=10)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [60, 400,  40, 40 , 90]  
flags = [1, 0,  0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [42]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #assert len(bought_list) > len(recommended_list)
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_list, k=5):
        
    # your_code
    # Лучше считать через скалярное произведение, а не цикл
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_list = np.array(prices_list)[:k]
    
    # определяем метки купленных товаров среди рекомендованных
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    # рассчитаем revenue от рекомендованных релеватных товаров скалярным произведением
    recommended_relevant_rev = np.dot(flags, prices_list)
    
    precision_at_k = recommended_relevant_rev/sum(prices_list)
    
    return precision_at_k
    


In [43]:
# Добавим цены на рекомендованные товары
prices_for_recommended = [100, 40, 100, 240, 25, 400, 180, 110, 220, 55, 190, 500, 80, 160, 300]

In [44]:
precision(recommended_list, bought_list)

0.2

In [45]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [46]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [47]:
money_precision_at_k(recommended_list, bought_list, prices_for_recommended, k=5)


0.6732673267326733

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [48]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    # your_code
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    recall_at_k = sum(flags)/len(bought_list)
    
    return recall_at_k


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    # your_code
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.where(np.isin(recommended_list, bought_list), 1, 0)
    
    recommended_relevant_rev = np.dot(flags, prices_recommended)
    
    money_recall_at_k = recommended_relevant_rev/sum(prices_bought)
    
    return money_recall_at_k

In [49]:
recall(recommended_list, bought_list)

0.5

In [50]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

In [54]:
# Добавим цены на купленные товары
prices_for_bought = list(np.random.randint(10,500,len(bought_list)))
# Заменим значения релевантных рекомедованных товаров на уже известные цены этих рекомендованных товаров
for i in np.where(np.isin(bought_list, recommended_list))[0]:  #возвращаем индексы релеватных рекомендованных элементов
    prices_for_bought[i] = prices_for_recommended[i]

In [55]:
prices_for_bought

[362, 45, 100, 240]

In [56]:
prices_for_recommended

[100, 40, 100, 240, 25, 400, 180, 110, 220, 55, 190, 500, 80, 160, 300]

---

# Метрики ранжирования

Если важен порядок рекомендаций. Подробнее можно почитать [здесь](https://habr.com/ru/company/econtenta/blog/303458/). Формулы в статье могут несколько отличаться от формул в лекции 

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [57]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [58]:
ap_k(recommended_list, bought_list, k=5)

0.75

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [107]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],  #id товаров юзер 1
                    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],] #id товаров юзер 2
bought_list = [[521, 32, 143, 991] # юзер 1
               , [146, 29]] # юзер 2

In [62]:
def map_k(recommended_list, bought_list, k=5, u=1):
    
    # your_code
    num_users = len(bought_list)
    result = np.average([ap_k(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return result

In [63]:
map_k(recommended_list, bought_list)

0.875

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


https://pytorch.org/docs/stable/index.html

In [64]:
import torch
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

In [65]:
ys_true

tensor([2, 0, 2, 2, 2])

In [66]:
ys_pred

tensor([0.2970, 0.6631, 0.2565, 0.3999, 0.4754])

In [67]:
# итоговый результат модели - отсортированный ys_pred
ys_pred

tensor([0.2970, 0.6631, 0.2565, 0.3999, 0.4754])

In [68]:
ys_pred = np.array(ys_pred)

In [69]:
ys_pred

array([0.29698342, 0.66310203, 0.25645292, 0.39993823, 0.47543162],
      dtype=float32)

In [92]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == "exp2":
        gain = 2 ** y_value - 1
    elif gain_scheme == "const":
        gain = y_value
    else:
        raise ValueError(f"{gain_scheme} method not supported, only exp2 and const.")
    return float(gain)

In [97]:
def dcg_at_k(recommended_list, bought_list, k=5):
    
    flags = np.isin(recommended_list[:k], bought_list)
    
    norm_coeffs = [1/(i + 1) if i <= 2 else 1/math.log2(i + 1) for i in range(k)]
    
    return np.dot(flags, norm_coeffs) /k


def ndcg_at_k(recommended_list, bought_list, k=5):
    
    ideal_dcg = dcg_at_k(recommended_list, recommended_list[:k], k)
    return dcg_at_k(recommended_list, bought_list, k) / ideal_dcg 

In [99]:
def ndcg_k(recommended_list, bought_list, k=5):

    flags = np.isin(recommended_list[:k], bought_list)
    
    #рассчитаем DCG    
    coeffs = [1/(i + 1) if i <= 2 else 1/math.log2(i + 1) for i in range(k)]
    
    dcg = np.dot(flags, coeffs)
    
    #Рассчитаем идеальный DCG
    ideal_dcg = sum([1/(i+1) if i<=2 else 1/math.log2(i+1) for i in range(k)])
    
    ndcg = dcg/ideal_dcg
    
    return ndcg

In [ ]:
# ndcg_at_k(recommended_list, bought_list, k=5)  не работает, не понимаю как сделать правильно

### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

[MRR](https://drive.google.com/file/d/1uMFKqWULFmehS0pJtvXKRygjYxQOPise/view?usp=sharing)

In [103]:
def reciprocal_rank(recommended_list, bought_list, k=5):
    rec_relevant_indices = np.nonzero(np.isin(recommended_list[:k], bought_list))[0]
    if len(rec_relevant_indices) == 0:
        return 0
    rr = 1/(rec_relevant_indices[0]+1) #т к расчет идет с 1 (в отличии от индексов, которые с нуля)
    return rr

def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    
    num_users = len(bought_list)
    mrr = np.average([reciprocal_rank(recommended_list[i], bought_list[i], k) for i in range(num_users)])
    
    return mrr

In [109]:
mean_reciprocal_rank(recommended_list, bought_list)

1.0

# 3. Связь бизнес-метрик, ML-метрик и функции потерь(loss)

- **loss** - то, что оптимизирует модель (RMSE - Root Mean Squared Error)
- **ML-метрика** - то, как мы измеряем качество на test (money precision@5)
- **Бизнес-метрика** - то, что хочет оптимизировать бизнес (выручка)

В идеале loss = ML-метрика = бизнес-метрика, но это возмонжо в очень редких случаях

Надеемся, что:  
    *Оптимизация loss --> рост ML-метрик --> рост бизнес-метрик*

### Пример: рекомендательная система товаров на сайте  
      
На сайте есть баннер, на котором мы можем разместить 5 товаров. Надо порекомендовать эти 5 товаров персонально каждому юзеру

*Шаг 1: Определим бизнес-метрику*  
Бизнес хочет максимизировать выручку --> бизнес-метрика - **Выручка**  

*Шаг 2: Разложим ее на составляющие*  
Выручка =   
       Средний чек * кол-во покупок =   
       Средний чек * (число юзеров * конверсия из захода на сайт в заказ) =
       Число юзеров * (Средний чек * конверсия из захода на сайт в заказ)
       
Рекомендательная система влияет только на (Средний чек * конверсия из захода на сайт в заказ).   
Хорошим приближением этого является **money precision@5** - ML-метрика

*Шаг 3: loss*  
Это существенно сложнее. Персонализированные ML-модели не умеют напрямую оптимизировать конверсию. Стандартный loss - **RMSE**. Можно для начала попробовать его. 

P.S. Если вы хотите приблизить RMSE к money precision@5, то можно посчитать weighted RMSE, где вес каждого наблюдения = стоимость товара.   
P.S.S. Про другие виды loss будет рассказано в курсе